# 410921202 資工四 林芷萱 電腦視覺 HW2

在這次的程式碼中我多次計算玩家面向的方向，但我得不出其結果，如下程式碼:  
繪製線條  
end_point = tuple(np.array(c_cyan) + 20 * normalized_vector)  
cv2.line(image, c_cyan, tuple(map(int, end_point)), (0, 255, 255), 2)  
這部分程式碼印出來的線條跟player面向方向有誤
最後我決定計算的角度採用計算玩家的質心到最近一個塔的質心之間的旋轉角度

In [1]:
import cv2
import numpy as np

image = cv2.imread('input1.png')

# 定義青色和紅色的範圍
lower_cyan = np.array([90, 200, 100]) 
upper_cyan = np.array([255, 255, 130])
lower_red = np.array([0, 0, 150]) 
upper_red = np.array([25, 25, 255])

cyan_mask = cv2.inRange(image, lower_cyan, upper_cyan)
red_mask = cv2.inRange(image, lower_red, upper_red)

# 尋找玩家(青色)的輪廓
contours_cyan, _ = cv2.findContours(cyan_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# 如果找到了青色區域的輪廓，則計算面積最大的輪廓的質心和方向
if contours_cyan:
    cnt_cyan = max(contours_cyan, key=cv2.contourArea)
    M_cyan = cv2.moments(cnt_cyan)
    cX_cyan = int(M_cyan["m10"] / M_cyan["m00"])
    cY_cyan = int(M_cyan["m01"] / M_cyan["m00"])

    # 找面向的方向
    approx_cyan = cv2.approxPolyDP(cnt_cyan, 0.01 * cv2.arcLength(cnt_cyan, True), True)
    c_cyan = (cX_cyan, cY_cyan)

    # 計算面向之向量
    c1, c2, c3 = approx_cyan[:3, 0]
    vector1 = np.array([c1[0] - c2[0], c1[1] - c2[1]])
    vector2 = np.array([c3[0] - c2[0], c3[1] - c2[1]])
    normalized_vector = vector2 - vector1
    normalized_vector = normalized_vector.astype(float)
    normalized_vector /= np.linalg.norm(normalized_vector)

    # 繪製線條
    end_point = tuple(np.array(c_cyan) + 20 * normalized_vector)
    # print(end_point)
    cv2.line(image, c_cyan, tuple(map(int, end_point)), (0, 255, 255), 2)

    # 尋找最近的shrine(紅色)
    contours_red, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours_red:
        nearest_shrine_distance = float('inf')
        nearest_shrine_center = None
        for contour in contours_red:
            M_red = cv2.moments(contour)
            if M_red["m00"] != 0:
                cX_red = int(M_red["m10"] / M_red["m00"])
                cY_red = int(M_red["m01"] / M_red["m00"])
                distance = np.sqrt((cX_red - cX_cyan)**2 + (cY_red - cY_cyan)**2)
                if distance < nearest_shrine_distance:
                    nearest_shrine_distance = distance
                    nearest_shrine_center = (cX_red, cY_red)

        # 計算旋轉角度
        vector = np.array(nearest_shrine_center) - np.array(c_cyan)
        angle = np.arctan2(vector[1], vector[0]) * 180 / np.pi # 使用 np.arctan2() 計算青色區域的質心到最近紅色區域的質心之間的旋轉角度
        
        # 繪製到最近的shrine的黃線
        cv2.line(image, c_cyan, nearest_shrine_center, (0, 255, 255), 2) # 使用 cv2.line() 繪製連接青色區域質心和最近紅色區域質心的線條
        # 顯示角度
        cv2.putText(image, f'{angle:.2f}', (1, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

cv2.imshow("Result", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
